In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using NearestNeighbors
using JLD
using Trees
using AbstractTrees
using BlossomV

In [2]:
ee = load("../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld","ee");


In [3]:
using Lumberjack
remove_truck("console")
add_truck(LumberjackTruck(STDOUT, "info", Dict{Symbol,Any}(:is_colorized => true)), "console")
add_truck(LumberjackTruck("semhuff_ipynb.log"), "file-logger")

  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:4
  likely near In[3]:4
  likely near In[3]:4


Lumberjack.LumberjackTruck(IOStream(<file semhuff_ipynb.log>),nothing,Dict{Any,Any}(Pair{Any,Any}(:is_colorized,false),Pair{Any,Any}(:uppercase,false)))

In [4]:
#run(`tail semhuff_ipynb.log`)

true

== (generic function with 142 methods)

In [78]:
get_paths(y) == get_paths(sol)

false

In [46]:
print_tree(STDOUT, y)

Trees.BranchNode with 2 children. data = "x1"
├─ Trees.BranchNode with 0 children. data = "L11"
└─ Trees.BranchNode with 2 children. data = "x12"
   ├─ Trees.BranchNode with 0 children. data = "L121"
   └─ Trees.BranchNode with 0 children. data = "L122"


In [79]:
#TODO make this a testcase

y == sol

true

In [9]:
midi_tree = ee.classification_tree#[1][1][1][1][1][1][1][1][1][1]
@time semtree = semhuff(midi_tree, ee.embedding, 30);
#print_tree(STDOUT, semtree)

2016-09-02T15:57:12.71 - info: semantically sorting level: 22
perfect matching with 6518 nodes and 195075 edges
    starting init...done [0.202 secs]. 10 trees
    .8.6.4.2.0.
done [0.225 secs]. 928 grows, 0 expands, 17 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]
2016-09-02T15:57:23.546 - info: semantically sorting level: 21
perfect matching with 32324 nodes and 969255 edges
    starting init...done [1.974 secs]. 68 trees
    .64.56.48.40.32.24.16.8.6.4.2.0.
done [2.230 secs]. 7170 grows, 4 expands, 99 shrinks
    expands: [0.001 secs], shrinks: [0.002 secs], dual updates: [0.001 secs]
2016-09-02T16:02:01.874 - info: semantically sorting level: 20
perfect matching with 34866 nodes and 1045515 edges
    starting init...done [2.440 secs]. 78 trees
    .64.56.48.40.32.24.16.8.6.4.2.0.
done [2.716 secs]. 6635 grows, 4 expands, 117 shrinks
    expands: [0.001 secs], shrinks: [0.003 secs], dual updates: [0.001 secs]
2016-09-02T16:07:30.687 - info: sem

Trees.BranchNode with 2 children. data = ""

In [ ]:
JLD.save("semtree.jld", "semtree", semtree)

In [ ]:
open("tree.txt","w") do fp
    print_tree(fp, semtree)
end

In [10]:
using Trees

In [33]:
reload("Trees")

In [34]:
function semhuff_initialize_AdaGram(semtree::Trees.BranchNode, dim::Integer, num_meanings::Integer; alpha::Float64=0.01, d::Float64=0.0)
    paths = Dict(node.data=> path for (node, path) in get_paths(semtree))
        
    codes = Dict(w=> convert(Vector{Int8}, oc - 1) for (w, oc) in leaves_of(semtree))
    dict = AdaGram.Dictionary(convert(Vector{AbstractString},collect(keys(paths))))
    freqs = Vector{Int64}(length(codes))
    huffman_outputs = Vector{AdaGram.HierarchicalOutput}(length(codes))
    for word in dict.id2word
        id = dict.word2id[word]
        freqs[id] = round(Int64,ee.distribution[word] * ee.corpus_size) #Todo: The math on this is not quiet right, because subsampling could have messed with the Corpus Size

        huffman_outputs[id] = AdaGram.HierarchicalOutput(codes[word], paths[word])
    end;
        
    vm = AdaGram.VectorModel(freqs, dim, num_meanings, alpha,d, huffman_outputs)
    vm, dict
end

semhuff_initialize_AdaGram (generic function with 2 methods)

In [35]:
vm, dict = semhuff_initialize_AdaGram(semtree, 300, 5; alpha=0.15, d=0.0);

LoadError: LoadError: MethodError: Cannot `convert` an object of type Array{UInt8,1} to an object of type AbstractString
This may have arisen from a call to the constructor AbstractString(...),
since type constructors fall back to convert methods.
while loading In[35], in expression starting on line 1

In [ ]:
inplace_train_vectors!(vm, dict, train_fn, window;
                       threshold=subsample, context_cut=context_cut,
					   epochs=epochs, init_count=initcount, sense_treshold=sense_treshold)

In [28]:
base_name = "semhuff_v1"
param_save_fn =  "../eval/models/adagram/$(base_name).params.jld"
output_fn = "../eval/models/adagram/$(base_name).adagram_model"#"file to save the model (in Julia format)"
@assert !isfile(output_fn)
@param_save param_save_fn begin
	nprocessors = nprocs()
	train_fn  =  "../eval/data/corpora/WikiCorp/tokenised_lowercase_WestburyLab.wikicorp.201004.txt" #"training text data"
	output_fn = output_fn #file to save the model (in Julia format)"
    semsimsource_fn = "../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld"

	window = 10 #"(max) window size" C in the paper
	min_freq  = 20 #"min. frequency of the word"
	remove_top_k = 0 #"remove top K most frequent words"
	dim  = 300 #"dimensionality of representations"
	prototypes = 5 #"number of word prototypes" T in the paper
	alpha = 0.15 #"prior probability of allocating a new prototype"
	d  = 0.0 #"parameter of Pitman-Yor process" D in paper
	subsample = 1e-5 #"subsampling treshold. useful value is 1e-5"
	context_cut  = true #"randomly reduce size of the context"
	epochs = 1 #"number of epochs to train"
	initcount = 1. #"initial weight (count) on first sense for each word"
	stopwords = Set{AbstractString}() #"list of stop words"
	sense_treshold = 1e-10 #"minimal probability of a meaning to contribute into gradients"
	save_treshold = 0.0 #"minimal probability of a meaning to save after training"
end

Paramaters -- saving to ../eval/models/adagram/semhuff_v1.params.jld
----------


In [29]:
inplace_train_vectors!(vm, dict, train_fn, window;
                       threshold=subsample, context_cut=context_cut,
					   epochs=epochs, init_count=initcount, sense_treshold=sense_treshold)

LoadError: LoadError: MethodError: no method matching inplace_train_vectors!(::AdaGram.VectorModel, ::AdaGram.Dictionary, ::String, ::Int64; threshold=1.0e-5, context_cut=true, epochs=1, init_count=1.0, sense_treshold=1.0e-10)
Closest candidates are:
  inplace_train_vectors!(!Matched::AdaGram.VectorModel, !Matched::AdaGram.Dictionary, ::AbstractString, ::Int64; batch, start_lr, context_cut, epochs, init_count, sense_treshold, log_path, threshold) at /home/ubuntu/.julia/v0.5/AdaGram/src/gradient.jl:114
while loading In[29], in expression starting on line 1

In [ ]:
import WordEmbeddings: NetworkType

type SemHuff <: NetworkType
    source::GenWordEmbedding
end

In [ ]:
using Training
using PooledElements

In [ ]:
function initialize_network!(embed::GenWordEmbedding, network_type::SemHuff)
    source_tree = network_type.source.classification_tree
    source_embeddings = network_type.source.embedding
    
    debug("Began SemHuff sorting")
    semtree = semhuff(source_tree, source_embeddings, 30);
    debug("Completed SemHuff sorting")
    debug("Began classification tree creation")
    embed.classification_tree = transform_tree(semtree, 
                            leaf_transform = word->word,
    internal_transform = dummy -> LinearClassifier(2,embed.dim))
    
    embed.codebook = Dict(leaves_of(classification_tree))
    debug("Completed SemHuff Bootstrapping")
    embed
end

In [ ]:
sem_ee = deepcopy(ee)
sem_ee.network_type = SemHuff(ee)
sem_ee.embedding = Dict(pstring(word)=>wv for (word, wv) in ee.embedding)
initialize_embedding(sem_ee,sem_ee.init_type)
initialize_network!(sem_ee,sem_ee.network_type)